In [8]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta

In [9]:
# finding latitude and longitudes of places

In [10]:
def get_latlong_more(*addresses):
    geolocator = Nominatim(user_agent="shree_app")
    latitude = []
    longitude = []
    address = []
    for loc in addresses:
        address.append(loc)
        location = geolocator.geocode(loc)
        latitude.append(location.latitude)
        longitude.append(location.longitude)

    loc_dict =  {'location': address,
                "latitude" : latitude, 
                "longitude" : longitude}
    loc_df = pd.DataFrame.from_dict(loc_dict)
    
    return(loc_df)

In [11]:
df = get_latlong_more('treptower park, berlin', 'zehlendorf, berlin', 'mitte, berlin', 'potsdamer platz, berlin')
len(df)

4

In [12]:
# extracting dates between from and to 

In [13]:
def get_dates(start_date, end_date):
    
    start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")


    delta = end_date_dt - start_date_dt  # returns timedelta


    #print(delta.days)
    day_list = []
    for i in range(delta.days + 1):
        day = (start_date_dt + timedelta(days=i)).date()
        day = day.strftime("%Y-%m-%d")
    
        day_list.append(day)
        
    return day_list
        


In [14]:
get_dates('2022-05-01', '2022-05-06')
    

['2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06']

In [15]:
#full function of picnic weather

In [108]:
def picnic_weather_full(dates, *places):
    
    weather_dict = []
    
    for day in dates:
        
        for loc in places:
        
            url = f"https://api.brightsky.dev/weather?lat={float(np.asarray(get_latlong_more(loc)['latitude'].values))}&lon={float(np.asarray(get_latlong_more(loc)['longitude'].values))}&date={day}"
            #print(url)
            r = requests.get(url).json()
            weather = r['weather']
            n = len(weather)
            avg_windspeed = np.mean([wind['wind_speed'] for wind in weather if wind['wind_speed'] != None])
            avg_temperature = np.mean([temp['temperature'] for temp in weather if temp['temperature'] != None])
            avg_sunshine = np.mean([sun['sunshine'] for sun in weather if sun['sunshine'] != None])
            avg_precipitation = np.mean([prec['precipitation'] for prec in weather if prec['precipitation'] != None])
            weather_dict.append({'date': day,
                                 'location': loc,
                                        'avg_windspeed': avg_windspeed,
                                        'avg_temperature' : avg_temperature,
                                        'avg_sunshine' : avg_sunshine,
                                        'avg_precipitation' : avg_precipitation})
            
            df = pd.DataFrame.from_dict(weather_dict)
            df_fill = df.fillna(0)
            
            df_new = df_fill[(df_fill['avg_windspeed']<= 30.0) &
            (df_fill['avg_temperature']<= 30.0) &
            (df_fill['avg_temperature']>= 15.0) &
            (df_fill['avg_sunshine']>= 20.0) &
            (df_fill['avg_precipitation']<= 0.05)]
            
            df_result = df_new[['date','location']]
            
            
        
    return df_result

In [109]:
date_list = get_dates('2022-05-12', '2022-05-19')

In [110]:
out = picnic_weather_full(date_list, 'pankow, berlin',  'zehlendorf, berlin', 'treptower park, berlin')

/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/shreetamakarmakar/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [111]:
out.head(20)

,date,location
10,2022-05-15,"zehlendorf, berlin"
14,2022-05-16,"treptower park, berlin"
19,2022-05-18,"zehlendorf, berlin"
20,2022-05-18,"treptower park, berlin"
21,2022-05-19,"pankow, berlin"
22,2022-05-19,"zehlendorf, berlin"
23,2022-05-19,"treptower park, berlin"
